# GPT GENERATED cases

In [1]:
import random
import math
import numpy as np
import time

from shapely.geometry import Polygon, box
from shapely.affinity import scale, translate
from shapely import wkt
from shapely.ops import transform
from pyproj import Transformer

import folium
from folium.features import DivIcon
from IPython.display import display, clear_output

## Helper Function

In [2]:
def displayOSM(i, polygon, bbox, area_percentage=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north-0.25, east],
        icon=DivIcon(
            icon_size=(350,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: #4400ff;">points '+str(i)+' | area '+str(area_percentage*100.0)+'% BBOX</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # return m

In [17]:
def get_bbox_polygon(bbox):
    minx, miny, maxx, maxy = bbox
    return box(minx, miny, maxx, maxy)

def create_regular_polygon(center_x, center_y, radius, num_points):
    """Generates a regular convex polygon"""
    return Polygon([
        (
            center_x + radius * math.cos(2 * math.pi * i / num_points),
            center_y + radius * math.sin(2 * math.pi * i / num_points)
        )
        for i in range(num_points)
    ])

def scale_to_bbox_area(geom, target_area):
    """Scales a geometry to match a target area"""
    current_area = geom.area
    scale_factor = math.sqrt(target_area / current_area)
    return scale(geom, xfact=scale_factor, yfact=scale_factor, origin='center')

def add_holes_to_polygon(poly, num_holes):
    """Adds holes inside a polygon by subtracting smaller shapes"""
    if not poly.is_valid or poly.is_empty:
        return poly
    minx, miny, maxx, maxy = poly.bounds
    holes = []
    for i in range(num_holes):
        hole_width = (maxx - minx) * 0.1
        hole_height = (maxy - miny) * 0.1
        hole_x = random.uniform(minx + hole_width, maxx - hole_width)
        hole_y = random.uniform(miny + hole_height, maxy - hole_height)
        hole = box(hole_x, hole_y, hole_x + hole_width, hole_y + hole_height)
        if poly.contains(hole):
            holes.append(hole.exterior.coords)
    return Polygon(shell=poly.exterior.coords, holes=holes)

# GENERATOR SUB FUNCTIONS

def genCase1(bbox, num_points, percentage_of_area=0.5):
    """Simple polygon with fixed area (50% of BBOX)"""
    bbox_poly = get_bbox_polygon(bbox)
    cx, cy = bbox_poly.centroid.x, bbox_poly.centroid.y
    radius = min(bbox_poly.bounds[2] - bbox_poly.bounds[0],
                 bbox_poly.bounds[3] - bbox_poly.bounds[1]) / 4
    poly = create_regular_polygon(cx, cy, radius, num_points)
    poly_scaled = scale_to_bbox_area(poly, bbox_poly.area * percentage_of_area)
    return poly_scaled.wkt

def genCase2(bbox, num_points=4, percentage_of_area=1.0):
    """Simple polygon with varying area (1%-100%)"""
    return genCase1(bbox, num_points, percentage_of_area)

def genCase3(bbox, holes=1):
    """Polygon with one hole"""
    bbox_poly = get_bbox_polygon(bbox)
    outer = scale(bbox_poly, 0.9, 0.9, origin='center')
    return add_holes_to_polygon(outer, holes).wkt

def genCase6(bbox, holes):
    """Polygon with multiple holes"""
    bbox_poly = get_bbox_polygon(bbox)
    outer = scale(bbox_poly, 0.9, 0.9, origin='center')
    return add_holes_to_polygon(outer, holes).wkt

def genCaseLineLike(bbox):
    """Polygon with very narrow width (line-like shape)"""
    minx, miny, maxx, maxy = bbox
    height = (maxy - miny) * 0.01
    return box(minx, miny, maxx, miny + height).wkt

## USER GLOBAL VARIABLES

In [4]:
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 10
NUM_OF_HOLES = 10
PERCENTAGE_OF_AREA = 0.5  # 50% area of INPUT_BBOX

## Cases

In [5]:
poly = genCase1(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
print(f"LINEAR POLYGON \n\n{poly}\n")
displayOSM(NUM_OF_POINTS, poly, INPUT_BBOX, PERCENTAGE_OF_AREA)

LINEAR POLYGON 

POLYGON ((11.649239089891907 48.15486111114964, 11.628747636018641 48.21792732138819, 11.575100313299522 48.25690438285726, 11.508788575607827 48.25690438285726, 11.455141252888708 48.21792732138819, 11.434649799015443 48.15486111114964, 11.455141252888708 48.0917949009111, 11.508788575607827 48.05281783944204, 11.575100313299522 48.05281783944204, 11.628747636018641 48.0917949009111, 11.649239089891907 48.15486111114964))



In [6]:
poly = genCase2(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
print(f"LINEAR POLYGON \n\n{poly}\n")
displayOSM(NUM_OF_POINTS, poly, INPUT_BBOX, PERCENTAGE_OF_AREA)

LINEAR POLYGON 

POLYGON ((11.649239089891907 48.15486111114964, 11.628747636018641 48.21792732138819, 11.575100313299522 48.25690438285726, 11.508788575607827 48.25690438285726, 11.455141252888708 48.21792732138819, 11.434649799015443 48.15486111114964, 11.455141252888708 48.0917949009111, 11.508788575607827 48.05281783944204, 11.575100313299522 48.05281783944204, 11.628747636018641 48.0917949009111, 11.649239089891907 48.15486111114964))



In [18]:
poly = genCase3(INPUT_BBOX, holes=1)
print(f"LINEAR POLYGON \n\n{poly}\n")
displayOSM(NUM_OF_POINTS, poly, INPUT_BBOX, PERCENTAGE_OF_AREA)

LINEAR POLYGON 

POLYGON ((11.705069444453809 48.070861111149576, 11.705069444453809 48.23886111114971, 11.378819444453548 48.23886111114971, 11.378819444453548 48.070861111149576, 11.705069444453809 48.070861111149576), (11.508601947462504 48.10870908343727, 11.508601947462504 48.12550908343728, 11.475976947462478 48.12550908343728, 11.475976947462478 48.10870908343727, 11.508601947462504 48.10870908343727))



In [19]:
poly = genCase6(INPUT_BBOX, holes=NUM_OF_HOLES)
print(f"LINEAR POLYGON \n\n{poly}\n")
displayOSM(NUM_OF_POINTS, poly, INPUT_BBOX, PERCENTAGE_OF_AREA)

LINEAR POLYGON 

POLYGON ((11.705069444453809 48.070861111149576, 11.705069444453809 48.23886111114971, 11.378819444453548 48.23886111114971, 11.378819444453548 48.070861111149576, 11.705069444453809 48.070861111149576), (11.630371086522581 48.108109874400355, 11.630371086522581 48.124909874400366, 11.597746086522555 48.124909874400366, 11.597746086522555 48.108109874400355, 11.630371086522581 48.108109874400355), (11.578483068886129 48.13058555036904, 11.578483068886129 48.14738555036905, 11.545858068886103 48.14738555036905, 11.545858068886103 48.13058555036904, 11.578483068886129 48.13058555036904), (11.584523558301523 48.15200671165043, 11.584523558301523 48.16880671165044, 11.551898558301497 48.16880671165044, 11.551898558301497 48.15200671165043, 11.584523558301523 48.15200671165043), (11.692427241167104 48.21703626949433, 11.692427241167104 48.23383626949434, 11.659802241167078 48.23383626949434, 11.659802241167078 48.21703626949433, 11.692427241167104 48.21703626949433), (11.54

In [20]:
poly = genCaseLineLike(INPUT_BBOX)
print(f"LINEAR POLYGON \n\n{poly}\n")
displayOSM(NUM_OF_POINTS, poly, INPUT_BBOX, PERCENTAGE_OF_AREA)

LINEAR POLYGON 

POLYGON ((11.723194444453823 48.06152777781623, 11.723194444453823 48.063394444482896, 11.360694444453532 48.063394444482896, 11.360694444453532 48.06152777781623, 11.723194444453823 48.06152777781623))

